In [84]:
import re
import sys
import os
import requests
import time
import csv
import ntpath
import pandas as pd
from bs4 import BeautifulSoup
from bs4.element import Comment
from urllib.parse import urlparse, urljoin
from os import path
from datetime import datetime
from pprint import pprint

from IPython.core.debugger import set_trace

"""
Changes:
1. Changed save_Summaries, also prints visited urls
2. Automatically creates a new folder under current directory and saves generated files in that folder instead
3. Improved exception handing, now also prints exception encountered
4. Squashed some bugs 
"""

'\nChanges:\n1. Changed save_Summaries, also prints visited urls\n2. Automatically creates a new folder under current directory and saves generated files in that folder instead\n3. Improved exception handing, now also prints exception encountered\n4. Squashed some bugs \n'

In [81]:
class CollegeCrawl:
    gap_Insecond=0
    max_Pages=5
    headers = {'User-Agent': 'Mozilla/5.0'}



    def __init__(self,_collegename, _rooturl, 
            _prioritykeywords=['apply', 'admission', 'application', 'deadline'], 
           # _url_file=None,            
            _save_to_folder=None,
            _respectrobottxt=True):
    
        self.college=_collegename
         
        if urlparse(_rooturl).scheme=="":
            print('URL needs to have scheme. Please try again.')
            raise Exception('URL needs to have scheme')            
            return
        self.rootUrl=_rooturl
        
        self.priorityKeywords=_prioritykeywords  
        
        toSaveDir = os.path.join(os.getcwd(),"collegeFiles")
        if not os.path.exists(toSaveDir):
            os.makedirs(toSaveDir)
        
        
        
        if _save_to_folder==None or path.isdir(_save_to_folder)==False:
            self.save_to_folder=toSaveDir
        else:
            self.save_to_folder=toSaveDir
            

        
        self.existingurlfile=path.join(self.save_to_folder,re.sub("\s+?", "_", self.college.strip()+'.csv'))
            
        self.respectRobottext=_respectrobottxt #currently not doing anything
        
        #self.allUrls=set()
        #self.allRankedUrls = [] 
        self.allurls={}
        #self.rejectedUrls=set() 
        #self.visitedUrls=set()
        self.scheme=urlparse(self.rootUrl).scheme
        self.base_domain=urlparse(self.rootUrl).netloc
        self.files=[]
    
    def __str__(self):
        return "************Crawl {}({})************".format(self.college, self.rootUrl)
    
        
    """
    def __getDomainFromUrl(self, url):
        #url = "http://www.mit.edu/"
        firstIndex = url.find('//')
        length = len(url)-1 if url.endswith('/') else len(url)      
        return url[firstIndex+2:length]
    """
    
    """
     Read college-visiting csv file and put the content to dictionary{url:status_code}
     return dictionary {url:status_code}
    """
    def __load_DiscoveredUrls(self, delimiter=',', hasHeader=False, header_names=['url', 'status_code']):   
        if self.existingurlfile==None:
            return {}
        else:
            if path.exists(self.existingurlfile) and re.sub('\s+?', '_', self.college) in ntpath.basename(self.existingurlfile):
                df_urls=pd.read_csv(self.existingurlfile,  delimiter=delimiter).iloc[:, 0:2]   
                header_row=list(df_urls.columns) 
            
                if re.match('^http', header_row[0]):
                    df_urls.columns=header_names
                    df_urls=pd.concat([df_urls, pd.DataFrame([header_row], columns=header_names)])                
                else: #even it is not, still assign the header_name
                    df_urls.columns=header_names 
                
                return dict(zip(df_urls['url'], df_urls['status_code'])) #format: url:status_code. i.e., url is the key
                #another options is: df_urls.to_dict('list') #format of {url:[url1, url2...], status_code:[0, 200...]}
            else:
                return {}        
    
    def __addAllUrlsInOnePage(self, soup):
        #soup=BeautifulSoup(response.text, 'html.parser') 
        for link in soup.find_all('a'): 
            if link.has_attr('href'):
                url=link['href']       
                url=re.sub(r'[\/_+!@#$?\\\s]+$', '', url)
                parsed_uri = urlparse(url) 
                absolute_url=''
           
                if (parsed_uri.netloc=='') and (parsed_uri.scheme=='') and re.match(r'^\/.*\w$', parsed_uri.path) :
                    absolute_url=urljoin(self.scheme+'://'+self.base_domain,parsed_uri.path)
                elif parsed_uri.netloc==self.base_domain and re.match('^http', parsed_uri.scheme):
                    absolute_url=url
                elif parsed_uri.netloc==self.base_domain and parsed_uri.scheme=="" and re.match(r'^\/.*\w$', parsed_uri.path):
                    absolute_url=self.scheme+'://'+parsed_uri.netloc+parsed_uri.path
                else:
                    continue 
                    
                if absolute_url!='' and absolute_url not in self.allurls:   
                    self.allurls[absolute_url]=0
                    
    def __get_Pagetext(self,soup):
        #soup = BeautifulSoup(body.text, 'html.parser') #another type is content which is byte (like image)
        texts = soup.findAll(text=True) 
        visible_texts = filter(self.__tag_visible, texts)       
    
        return [ [t.parent.name,   
             t.parent.previousSibling.name if t.parent.previousSibling!=None else None, 
             t.nextSibling.name if t.nextSibling!=None else None,
             re.sub(r'[\s+\t]',' ',t) ]  for t in visible_texts if len(t.strip())>2] 
    
    def __tag_visible(self,element):
        if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
            return False
        if isinstance(element, Comment):
            return False
        return True 
     
    
    
    '''
        save from resonse. called in the initial loop
    '''
    def __saveToCsv_FromResponse(self, url, soap):
        content= self.__get_Pagetext(soap)
        self.__saveToCsv(url, content) 
        
    '''
        save to csv file and append it to self.files
    '''    
    def __saveToCsv(self, url, content):
        filename=url.replace('.', '_dot_').replace('/', '_').replace('?', '_').replace(':', '_')+'_'+datetime.now().strftime("%m_%d_%Y_%H_%M_%S")+'.csv'
        fullname=path.join(self.save_to_folder, filename)
        try:
            with open(fullname, 'w', newline='', encoding='utf-8') as csvfile:
                writer = csv.writer(csvfile, delimiter='\t', quoting=csv.QUOTE_MINIMAL)
                writer.writerow(['url', 'parent', 'ps', 'ns', 'text'])
                for lll in content: 
                    lll.insert(0, url)
                    writer.writerow(lll)  
            self.files.append(fullname) 
        except IOError as e:
            print('failed to save file.')
            print("Exception is: ", e)
            
    def __save_url_file(self):
         #save file as well  
            try:
                with open(self.existingurlfile, 'w', newline='', encoding='utf-8') as csvfile:
                    writer = csv.writer(csvfile, delimiter=',', quoting=csv.QUOTE_MINIMAL)
                    writer.writerow(['url','status_code'])
                    for url, status_code in self.allurls.items():  
                        writer.writerow([url, status_code])  
            except IOError:
                print('IO Error in saving summaries')
 

    """
        get all urls starting from rootUrl
    """
    def getAllUrls(self):  
        self.allurls=self.__load_DiscoveredUrls() 
        if len(self.allurls)==0:
            self.allurls={self.rootUrl:0}  
            unvisited=[self.rootUrl]
        else:
            unvisited=[url for url, status_code in self.allurls.items() if status_code==0]
            # not necessary if priority words do not change
            if not unvisited:
                unvisited=[self.rootUrl]
        
        pages_visited =0
        try:  
            while unvisited:
                if pages_visited==CollegeCrawl.max_Pages:
                    break 
                pages_visited+=1         
                    
                unvisited=sorted(unvisited, key=lambda item: (sum([w in item for w in self.priorityKeywords])*100+10)/len(item))
                url=unvisited.pop()  
                response=requests.get(url, headers=CollegeCrawl.headers)     
                status_code=response.status_code
                self.allurls[url]=status_code
                
                # save all urls in the page to allurls dictionary with status code 0
                contentType = response.headers.get('content-type')    
                if status_code==200 and 'text/html' in contentType:
                    soup=BeautifulSoup(response.text, 'html.parser') 
                    
                    self.__addAllUrlsInOnePage(soup)
                    
                    # write the page to csv
                    self.__saveToCsv_FromResponse(url, soup)     
             
                unvisited=[name for name, code in self.allurls.items() if code==0]
                time.sleep(CollegeCrawl.gap_Insecond) #wait for few seconds. 
          
        except Exception as e: 
            print('url "{}" went wrong'.format(url))  
            print(e)
            self.allurls[url]=999
                
        
   
           
    """
        display summaries 
    """
    
    def save_Summaries(self):
        if self.allurls:
            df_urls=pd.DataFrame(list(self.allurls.items()), columns=['url', 'status_code'])
            print(self)
            
            print("Summary of status code for URLs so far")
            print(df_urls.groupby('status_code').count().reset_index())
            print('\n')
            print("Visited URLs so far, including previously visited URLs")
            visited=[name for name, code in self.allurls.items() if code!=0]
            print(visited)
            
            
            self.__save_url_file()
                
        if self.files:
            print('\nThe following {} file(s) are generated this run. '.format(len(self.files)))
            pprint(self.files)    
            print('\n')

    """
        Save One Page
        filename, if not None, should not be full name. use import ntpath ntpath.basename("a\b\c")
    """
    def SaveToCsv_FromUrl(self,url): # tab delimiter only 
        url=url.strip() 
        content=self.Read_Oneurl(url) #in format of (a,a,a,a)   
        self.SaveToCsv(url, content)
        #return SaveToCsv_FromResponse()
    
    
    def setMaxpage(self, maxPages):
        CollegeCrawl.max_Pages = maxPages
        
    def setGap(self, gap):
        CollegeCrawl.gap_Insecond = gap
    
    
    def crawl(self):
        self.getAllUrls()
        self.save_Summaries()
        
     
            
        

    
   

   

In [83]:
colleges = [["UCLA", "http://www.ucla.edu"], 
           ["Stanford", "https://www.stanford.edu/"],
           ["Yale", "https://www.yale.edu/"],
           ["UW", "http://www.washington.edu/"]] 
for college in colleges:
    cr = CollegeCrawl(college[0], college[1])
    cr.crawl()


************Crawl UCLA(http://www.ucla.edu)************
Summary of status code for URLs so far
   status_code  url
0            0   65
1          200   10


Visited URLs so far, including previously visited URLs
['http://www.ucla.edu', 'http://www.ucla.edu/admission', 'http://www.ucla.edu/admission/undergraduate-admission', 'http://www.ucla.edu/admission/graduate-admission', 'http://www.ucla.edu/admission/extension-enrollment', 'http://www.ucla.edu/admission/international-admission', 'http://www.ucla.edu/admission/transfer-admission', 'http://www.ucla.edu/admission/affordability', 'http://www.ucla.edu/visit', 'http://www.ucla.edu/apply']

The following 5 file(s) are generated this run. 
['C:\\Users\\Kentt\\stempro\\hw\\y2019dec29\\collegeFiles\\http___www_dot_ucla_dot_edu_admission_graduate-admission_01_02_2020_19_36_11.csv',
 'C:\\Users\\Kentt\\stempro\\hw\\y2019dec29\\collegeFiles\\http___www_dot_ucla_dot_edu_admission_extension-enrollment_01_02_2020_19_36_11.csv',
 'C:\\Users\\Kentt

C:\Users\Kentt\stempro\hw\y2019dec29\collegeFiles


In [ ]:
def getResponse(url):
    headers = {'User-Agent': 'Mozilla/5.0'}
    response = requests.get(url)
    #response = requests.get(url,None, headers=headers)
    print("response status code for URL {} is {}".format(url,response.status_code))
    print("request headers for URL {} is {}".format(url, response.request.headers))
    
    
    #help(response)
    #help(requests.get)
    
    print(response.text)
 


getResponse("https://www.mit.edu")
#getResponseCode("http://www.stanford.edu")
#getResponseCode("http://www.harvard.edu")

In [ ]:
%debug

In [ ]:
_ih[-5:]

C:\Users\Kentt\stempro\hw\y2019dec29\collegeFiles
